In [64]:
#All packages

import geopandas as gpd
import pandas as pd
from shapely import geometry
from shapely.geometry import Polygon, LineString, Point
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt, mpld3
from mpld3 import fig_to_html, plugins
import random


In [65]:
#main variables 

directory = "./archives"
ext = ".geojson"

#variable that contains all the searched filters
lista=[]
plt.rcParams['figure.figsize'] = (20, 10) #setting parameters
listadirs=[]
dic={}
filtrolist=[]
fp={}


df_places = gpd.read_file('./Lisboa.geojson') #read file Lisboa.geojson

In [66]:
#this section of the code is useless to the aplication wep
#function to search
def leitura(filtro = "NULL"):
    i=0
    sum=0
    global lista, listadirs, dic, filtrolist
    listafiles=[]


    for root, dirs, files in os.walk(directory):
        
        if len(dirs) > 0:
            listadirs = dirs

        if len(files) > 0:
            for value in files:
                listafiles.append(root.replace("/", "\\")+"\\"+value.replace("/","\\"))

            dic[listadirs[i]] = listafiles
            listafiles = []
            i+=1    
        
        for file in files:
            if file.endswith(ext):
                if filtro is not None:
                    if filtro.find(file.replace(".geojson","")) != -1 and len(file) == len(filtro):                   
                        ficheiro = os.path.join(root, file)
                        filtrolist.append(filtro)
                        lista.append(gpd.read_file(ficheiro))


In [67]:
#this section of the code is useless to the aplication wep
#code with the objective os input severals directory
leitura('Feiras.geojson')
leitura('Escolas 1_Ciclo.geojson')
leitura('Hospitais Publicos.geojson')
leitura('Administracao_Publica.geojson')
leitura('Estacoes Fluviais.geojson')
leitura('Mercados.geojson')
leitura('Farmacias.geojso')
leitura('Hospitais Publicos.geojson')
leitura('Estacoes Metro.geojson')
leitura('Desporto_Instalacoes.geojson')

In [68]:
#this section of the code is useless to the aplication wep
#creation of a dictionary with all the importance quantities for each file
amount = []
importance = {}

for a in range(len(filtrolist)):
    
    amount.append(5)
    importance[filtrolist[a]]=amount[a]

In [73]:
#this section of the code is useless to the aplication wep
inp = int(input("Selecione o modo:\n1-Padrao    2-Satelite"))

if inp == 1:
    mode = True
elif inp == 2:
    mode = False
else: mode = 0


In [70]:
'''Choropleth with Points of Filter'''

geopoints = []
geopointsfiltered = []
geopointsforexplore = {}
count = []
total = []


#organizing the names
df_places.NOME=sorted(df_places.NOME)


# main loop for processing filtered elements
for b in range(len(lista)):
    # Loop for appending geometry per each filter
    for idx, row in lista[b].iterrows():
        geopoints.append(row.geometry)

    # for each non empty element inside geopoints save it to list geopointsfiltered      
    for ele in geopoints:
        if ele != None:
            geopointsfiltered.append(ele)

    #Nested FOR loop to analize the intersection of points with regions 
    # in this for loop we check each filter 
    for a in geopointsfiltered:
        # in this for loop we check each region
        for idx1, row1 in df_places.iterrows():
                # if the point is inside of the geometry that defines the region then we append to the list count
                if a.intersects(row1.geometry) == True:
                    count.append(row1.NOME) #append em count da repetição das localidades


    #for loop to analize the number of times that points appear in each region to later append them to a "total".
    for a in df_places.NOME:
        numb = count.count(a)
        total.append(numb) 

    #Convert to float         
    for a in range(len(total)):
        total[a] = float(total[a])

    
    fp[filtrolist[b]] = total
    # Reset lists to later be used by other filters
    geopointsforexplore[filtrolist[b]] = geopointsfiltered
    geopoints = []
    geopointsfiltered= []
    count = []
    total = []

In [71]:
# variable with unknown uses
sum=0
#organizing the names of Lisboa 
df_places.NOME=sorted(df_places.NOME)

#add up all the points and then divide by the total amount 
for a in range(len(filtrolist)):
    for b in range(0, len(fp[filtrolist[a]])):
        # add individual count of each point to later get the a total value of all points, ex : filters happen 50 + 30 + 20 times
        sum+=fp[filtrolist[a]][b]
    for b in range(0, len(fp[filtrolist[a]])): 
        # divide each point by the total of points 
        fp[filtrolist[a]][b] = (fp[filtrolist[a]][b] / sum) * importance[filtrolist[a]]
    # reset sum for next filter
    sum=0 



choropleth=[]
#sum of the points of the same regions 
for a in range(len(fp[filtrolist[0]])):
    # flatten array
    for b in range(0, len(filtrolist)):
        sum += fp[filtrolist[b]][a]
    
    choropleth.append(sum)
    sum=0
 
# add to variable inside geojson
df_places["Censos 2021 População Lisboa_field_4"] = choropleth


In [76]:
if mode==True:
    fig, ax=plt.subplots()
    c=0
    for a in lista:
        c+=1
        a.plot(ax=ax, zorder=c)

    # print map
    axis = df_places.plot(ax=ax, edgecolor = "black", column = "Censos 2021 População Lisboa_field_4", cmap= "YlOrRd", zorder=1)

    for idx, row in df_places.iterrows():
        coord = row['geometry'].centroid
        coordinates = coord.coords.xy

        x, y = coordinates[0][0], coordinates[1][0]
        axis.annotate(row['NOME'], xy=(x, y), xytext=(x, y), font ="Tahoma")  

    #print(filtrolist)
    html_str = mpld3.fig_to_html(fig)
    Html_file= open("index.html","w")
    Html_file.write(html_str)
    Html_file.close()

    

elif mode == False:
    color = []

    for i in range(len(lista)):
        color.append('#%06X' % random.randint(0, 0xFFFFFF))

    plotexplore = df_places.explore(column = "Censos 2021 População Lisboa_field_4", 
                        cmap='YlOrRd', style_kwds=dict(color = "black"))


    for a in range(len(lista)):
        axa=random.randint(0,len(color)-1)
        lista[a] = lista[a][lista[a].geometry != None]
        d = {'name': filtrolist[a].replace('.geojson', ''), 'geometry': lista[a].geometry}
        gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
        gdf.explore(m=plotexplore, 
                    color = color.pop(axa),  
                    style_kwds=dict(weight=5),
                    popup=True)


    plotexplore.save("index.html")